# Step 21 Operator Overloading(2)

## 21.1 Using with ndarray

If __obj__ is the instance of Variable, then leave it as it is. <br>
If __obj__ is not the instance of Variable, then assume that it is Variable.

In [28]:
def as_variable(obj):
    if isinstance(obj, Variable):
        return obj
    return Variable(obj)


Convert all elements(x) in inputs to Variable instance. 

In [29]:
class Function:
    def __call__(self, *inputs):
        inputs=[as_variable(x) for x in inputs] #added
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]
        
        if Config.enable_backprop:
            self.generation=max([x.generation for x in inputs])         
            for output in outputs:
                output.set_creator(self)
            self.inputs = inputs
            self.outputs=[weakref.ref(output) for output in outputs] 
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

In [50]:
%run ../Codes/step21.ipynb

In [51]:
x=Variable(np.array(2.0))
y=x+np.array(3.0)
print(y)

variable(5.0)


## 21.2 Using with float, int

In [52]:
def add(x0, x1):
    x1=as_array(x1)
    return Add()(x0,x1)

In [53]:
x=Variable(np.array(2.0))
y=x+3.0
print(y)

variable(5.0)


Now operation between float and Variable can be done. 

## 21.3 Problem 1: First arguement is float or int

Since the Variable is the second arguement, we have to implement the __rmul__ method.

In [55]:
x=Variable(np.array(2.0))
y=3.0*x+1.0
print(y)

variable(7.0)


## 21.4 Problem 2: First arguement is ndarray and second is Variable

In [56]:
x=Variable(np.array([1.0]))
y=np.array([2.0])+x

We have to set the priority so that __radd__ method of __Variable__ is called instead of __add__ method of ndarray instance. 